# Vulture

## Advanced Use Cases of Vulture Text Pre-Processing 

In [1]:
import os
import pickle
import pathlib
import pandas as pd

from TELF.pre_processing import Vulture

from TELF.pre_processing.Vulture.modules import SimpleCleaner
from TELF.pre_processing.Vulture.modules import LemmatizeCleaner
from TELF.pre_processing.Vulture.modules import SubstitutionCleaner
from TELF.pre_processing.Vulture.modules import RemoveNonEnglishCleaner

from TELF.pre_processing.Vulture.default_stop_words import STOP_WORDS
from TELF.pre_processing.Vulture.default_stop_phrases import STOP_PHRASES

## 0. Load Dataset

### Input

In [2]:
DATA_DIR = os.path.join('..', '..', 'data')
DATA_DIR = pathlib.Path(DATA_DIR).resolve()

In [3]:
DATA_FILE = 'documents.p'

In [4]:
documents = pickle.load(open(os.path.join(DATA_DIR, DATA_FILE), 'rb'))
len(documents)

9

### Output

In [5]:
RESULTS_DIR = 'results'
RESULTS_DIR = pathlib.Path(RESULTS_DIR).resolve()

In [6]:
RESULTS_FILE = 'clean_documents.p'

In [7]:
try:
    os.mkdir(RESULTS_DIR)
except FileExistsError:
    pass

### Examine Data Format

In [8]:
# key serve as document unique ids
list(documents.keys())

['ad68055e-677f-11ee-95d4-4ab2673ea3f0',
 'ad680626-677f-11ee-95d4-4ab2673ea3f0',
 'ad680658-677f-11ee-95d4-4ab2673ea3f0',
 'ad680680-677f-11ee-95d4-4ab2673ea3f0',
 'ad6806a8-677f-11ee-95d4-4ab2673ea3f0',
 'ad6806d0-677f-11ee-95d4-4ab2673ea3f0',
 'ad6806f8-677f-11ee-95d4-4ab2673ea3f0',
 'ad680716-677f-11ee-95d4-4ab2673ea3f0',
 'ad68073e-677f-11ee-95d4-4ab2673ea3f0']

In [9]:
# values are the text that needs to be cleaned
documents[next(iter(documents))]

'Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected 

## 1. Creating a Custom Pipeline

The cleaning modules can be arranged in a custom order specified by the user. They will be executed in the order that they appear. The settings of each component can be modified by changing the settings of the associated object in the list. The current supported modules are:

    - SimpleCleaner
    - RemoveNonEnglishCleaner
    - SubstitutionCleaner
    - LemmatizeCleaner

Here is an example of a custom pipeline. This pipeline is used to first remove non english text, apply simple cleaning, and then apply lemmatization.

In [10]:
vulture = Vulture(n_jobs  = 1, 
                  verbose = 10,  # Disable == 0, Verbose >= 1
                 )

In [11]:
steps = [
    RemoveNonEnglishCleaner(ascii_ratio=0.9, stopwords_ratio=0.25),
    SimpleCleaner(stop_words = STOP_WORDS,
                  stop_phrases = STOP_PHRASES,
                  order = [
                      'standardize_hyphens',
                      'isolate_frozen',
                      'remove_copyright_statement',
                      'remove_stop_phrases',
                      'make_lower_case',
                      'remove_formulas',
                      'normalize',
                      'remove_next_line',
                      'remove_email',
                      'remove_()',
                      'remove_[]',
                      'remove_special_characters',
                      'remove_nonASCII_boundary',
                      'remove_nonASCII',
                      'remove_tags',
                      'remove_stop_words',
                      'remove_standalone_numbers',
                      'remove_extra_whitespace',
                      'min_characters',
                  ]
                 ),
    LemmatizeCleaner('spacy'),
]

In [12]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/3 [00:00<?, ?it/s][Vulture]: Running RemoveNonEnglishCleaner module

100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 1342.85it/s]
[Vulture]: Running SimpleCleaner module

 67%|██████████████████████████████               | 2/3 [00:00<00:00, 15.63it/s][Vulture]: Running LemmatizeCleaner module

100%|█████████████████████████████████████████████| 3/3 [00:03<00:00,  1.05s/it]

CPU times: user 2.09 s, sys: 779 ms, total: 2.87 s
Wall time: 3.17 s


In [13]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected

## 2. Customizing Components

### SimpleCleaner

#### Setting Order of Operations

For the default patterns, the ```order``` parameter can be used to specify the order of operations within the SimpleCleaner component. The following examples shows how 5 simple operations can be applied in sequential order.

In [14]:
steps = [
    SimpleCleaner(stop_words = STOP_WORDS,
                  stop_phrases = STOP_PHRASES,
                  order = [
                      'make_lower_case',
                      'remove_()',
                      'remove_stop_words',
                      'remove_extra_whitespace',
                      'min_characters',
                  ]
                 ),
]

In [15]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running SimpleCleaner module

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.08it/s]

CPU times: user 80.5 ms, sys: 7.57 ms, total: 88 ms
Wall time: 83.6 ms


In [16]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected

#### Setting Min Number of Characters

The number of minimum characters to keep per token is a class attribute. This example shows how it can be changed.

In [17]:
steps = [
    SimpleCleaner(min_characters = 12,
                  order = ['min_characters']
                 ),
]

In [18]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running SimpleCleaner module

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 109.65it/s]

CPU times: user 12.7 ms, sys: 6.29 ms, total: 19 ms
Wall time: 14.3 ms


In [19]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected

#### Custom RegEx

The SimpleCleaner component allows the user to specify custom regular expressions patterns. This enables Vulture to be applied to specific datasets while maintaining the parallel and distributed capabilities. In the following example, we will define a simple regular expression to detect and remove acronyms in the text. An acronym will be assumed to be a series of one or more letters enclosed in parentheses. 

In [20]:
custom_patterns = {
    'remove_acronyms': (
        r"\s*\([A-Z]+\)\s*",  # pattern to detect uppercase letters enclosed by parentheses
        ' *** '               # the string to replace matches with. 
                              # the stars are used for the example to make the pattern stand out. 
                              # to fully remove the acronyms, an empty string can be used
    )
}

In [21]:
steps = [
    SimpleCleaner(order = ['remove_acronyms'],  # only perform the custom operation
                  custom_patterns = custom_patterns
                 ),
]

In [22]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running SimpleCleaner module

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 110.09it/s]

CPU times: user 13.6 ms, sys: 6.21 ms, total: 19.9 ms
Wall time: 15.7 ms


In [23]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected

### RemoveNonEnglishCleaner

This component is used to detect and remove non-english text from the dataset. The detection works through a simple heuristics. This heuristic uses two components: the ratio of ASCII characters to all characters and the ratio of stopwords to all words. While not flawless, this heuristic (with appropriate hyperparameters) provides a quick way to filter out foreign language documents.  

##### Define a placeholder Spanish document for Non-English detection

In [24]:
# This placeholder text tells a short story about a young girl named Ana who lives in a small 
# town surrounded by mountains and finds an old book filled with stories of adventure and magic.
foreign_text = 'En un pequeño pueblo rodeado de montañas, vivía una joven llamada Ana. Todos los días, ' \
               'ella caminaba por los senderos del bosque, disfrutando de la tranquilidad y la belleza de ' \
               'la naturaleza. Un día, mientras exploraba una parte del bosque que no conocía, Ana encontró ' \
               'un antiguo libro cubierto de polvo y hojas. Curiosa, lo abrió y comenzó a leer. Las páginas ' \
               'contenían historias de aventuras, magia y lugares lejanos. Ana se sintió inspirada por estas ' \
               'historias y decidió que un día ella también viviría sus propias aventuras.'

In [25]:
foreign_documents = {0: foreign_text}

In [26]:
steps = [
    RemoveNonEnglishCleaner(ascii_ratio=0.9, stopwords_ratio=0.25),
]

In [27]:
%time cleaned_documents = vulture.clean(foreign_documents, steps=steps)

[Vulture]: Cleaning 1 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running RemoveNonEnglishCleaner module

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 119.70it/s]

CPU times: user 11.5 ms, sys: 5.73 ms, total: 17.2 ms
Wall time: 13.3 ms


In [28]:
key = next(iter(foreign_documents))
foreign_documents[key], cleaned_documents[key]

('En un pequeño pueblo rodeado de montañas, vivía una joven llamada Ana. Todos los días, ella caminaba por los senderos del bosque, disfrutando de la tranquilidad y la belleza de la naturaleza. Un día, mientras exploraba una parte del bosque que no conocía, Ana encontró un antiguo libro cubierto de polvo y hojas. Curiosa, lo abrió y comenzó a leer. Las páginas contenían historias de aventuras, magia y lugares lejanos. Ana se sintió inspirada por estas historias y decidió que un día ella también viviría sus propias aventuras.',
 '')

### SubstitutitionCleaner

Substitutions allow Vulture to find specific substrings in the text and replaces them with some other substring. The intent of this component is to provide a Subject Matter Expert (SME) the ability to  normalize important terms in the vocabulary. Multiple terms can be used to represent the same concept. Alternatively, multiple tokens can represent a single term, the meaning of which will be destroyed through tokenization. Substitution can normalize these terms and also signal to Vulture that these terms do not require cleaning. This means that this component can also be used to "freeze" terms through a reflective substitution, leaving them unchanged throughout the cleaning process.

Substitution can be added as argument to the clean function. This will perform substitution before and after cleaning the text. However, the SubstitutionCleaner can be used like any of the other components as part of the pipeline. This finer-grained control allows the user to specify when substitution occurs and how the substitution map is handled. The SubstitutionCleaner has three attributes that control how the substitution map is processed

In [29]:
substitution_map = {
    'Foo Bar Bat': 'foo_bar',
    'testing': 'test'
}

#### Permute

This attribute permutes the substitution map such that whitespace is replaced with hyphens and vice versa

In [30]:
sub = SubstitutionCleaner(substitution_map, permute=True)
sub.substitution_map

{'Foo Bar Bat': 'foo_bar',
 'Foo-Bar Bat': 'foo_bar',
 'Foo Bar-Bat': 'foo_bar',
 'testing': 'test',
 'Foo-Bar-Bat': 'foo_bar'}

#### Lower 

This attribute converts the keys of the substitution map to lowercase

In [31]:
sub = SubstitutionCleaner(substitution_map, lower=True)
sub.substitution_map

{'Foo Bar Bat': 'foo_bar', 'foo bar bat': 'foo_bar', 'testing': 'test'}

#### Lemmatize 

This attribute lemmatizes the keys of the substitution map

In [32]:
sub = SubstitutionCleaner(substitution_map, lemmatize=True)
sub.substitution_map

{'Foo Bar Bat': 'foo_bar', 'testing': 'test', 'test': 'test'}

#### All Together

In [33]:
sub = SubstitutionCleaner(substitution_map, permute=True, lower=True, lemmatize=True)
sub.substitution_map

{'Foo Bar Bat': 'foo_bar',
 'foo bar bat': 'foo_bar',
 'Foo-Bar Bat': 'foo_bar',
 'Foo Bar-Bat': 'foo_bar',
 'foo-bar bat': 'foo_bar',
 'foo bar-bat': 'foo_bar',
 'testing': 'test',
 'test': 'test',
 'Foo-Bar-Bat': 'foo_bar',
 'foo-bar-bat': 'foo_bar'}

### LemmatizeCleaner

#### Apply NLTK Lemmatization

In [34]:
steps = [
    LemmatizeCleaner('nltk')
]

In [35]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running LemmatizeCleaner module

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.29s/it]

CPU times: user 1.17 s, sys: 107 ms, total: 1.28 s
Wall time: 1.3 s


In [36]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected

#### Apply Spacy Lemmatization

In [37]:
steps = [
    LemmatizeCleaner('spacy')
]

In [38]:
%time cleaned_documents = vulture.clean(documents, steps=steps)

[Vulture]: Cleaning 9 documents
  0%|                                                     | 0/1 [00:00<?, ?it/s][Vulture]: Running LemmatizeCleaner module

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.28s/it]

CPU times: user 1.73 s, sys: 487 ms, total: 2.22 s
Wall time: 2.28 s


In [39]:
key = next(iter(documents))
documents[key], cleaned_documents[key]

('Supervisory Control and Data Acquisition (SCADA) systems often serve as the nervous system for substations within power grids. These systems facilitate real-time monitoring, data acquisition, control of equipment, and ensure smooth and efficient operation of the substation and its connected devices. As the dependence on these SCADA systems grows, so does the risk of potential malicious intrusions that could lead to significant outages or even permanent damage to the grid. Previous work has shown that dimensionality reduction-based approaches, such as Principal Component Analysis (PCA), can be used for accurate identification of anomalies in SCADA systems. While not specifically applied to SCADA, non-negative matrix factorization (NMF) has shown strong results at detecting anomalies in wireless sensor networks. These unsupervised approaches model the normal or expected behavior and detect the unseen types of attacks or anomalies by identifying the events that deviate from the expected